In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from IPython.display import SVG
from graphviz import Source
from sklearn import tree
%matplotlib inline
from sklearn.metrics import classification_report, confusion_matrix


# Read Data & Review

In [ ]:
df = pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.nunique()

# Arrange Group columns more than 10 unique

In [ ]:

def age_values(age_values):
  # refer https://www.beresfordresearch.com/age-range-by-generation/
  if age_values < 9 :
    return 'youth (< 9Y)'
  elif 9 <= age_values < 25:
    return 'gen_z (9-25Y)'
  elif 25 <= age_values < 41:
    return 'Millennials (25-41Y)'
  elif 41 <= age_values < 57:
    return 'gen_x (41-57Y)'
  elif 57 <= age_values <67:
    return 'boomers_ii (57-67Y)'
  elif 67 <= age_values <76:
    return 'boomer_i (67-76Y)'
  elif 76 <= age_values < 94:
    return 'post_war (76-94Y)'
  elif age_value >= 94:
    return 'ww_ii (94Y <)'
  else:
    return 'none'

def glucose_values(glucose_values):
  #refer https://www.emedihealth.com/manage-diabetes.html
    if glucose_values < 53 :
      return 'server_hypoglycemia (< 53)'
    elif 53 <= glucose_values < 70:
      return 'hypoglycemia (53-70)'
    elif 70 <= glucose_values < 125:
      return 'normal (70-125)'
    elif 125 <= glucose_values < 200:
      return 'high (125-200)'
    elif  glucose_values > 200:
      return 'metabolic_consequences (200 <)'
    else:
      return 'none'

def bmi_values(bmi_values):
  #refer https://www.cdc.gov/healthyweight/assessing/bmi/adult_bmi/index.html

    if bmi_values < 18.5 :
      return 'underweight (< 18.5)'
    elif 18.5 <= bmi_values < 25:
      return 'normal (18.5-25)'
    elif 25 <= bmi_values < 30:
      return 'overweight (25-30)'
    elif  bmi_values >= 30:
      return 'obese 30 <'
    else:
      return 'none'


df['age_class']=df['age'].apply(age_values)
df['glucose_class']=df['avg_glucose_level'].apply(glucose_values)
df['bmi_class']=df['bmi'].apply(bmi_values)

df.columns

Now there are 3 columns, have been added!
* age_class
* gluclose_class
* bmi_class

# Review overall data

In [ ]:
ColumnName = ['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'age_class', 'glucose_class', 'bmi_class' ]


for i in range(len(ColumnName)):
    list_df = pd.DataFrame({''f'{ColumnName[i]}':   (df[ColumnName[i]].value_counts()) ,
                          '% Samples  ': round(100* (df[ColumnName[i]].value_counts())/(sum(df[ColumnName[i]].value_counts())),2),
                          'from total': sum(df[ColumnName[i]].value_counts())
    })
    print("")
    print(list_df)
    

    if (not np.issubdtype(type(ColumnName[i]), np.number)):
      valueCounts = df[ColumnName[i]].value_counts()
      valueCounts.plot.bar()
            
    else: 
      df[ColumnName].hist()
    
    plt.ylabel('Counts')
    plt.title(f'{ColumnName[i]}')
    plt.show()

# Review all stroke patients

In [ ]:
df_stroke = df[df['stroke'] == 1]

for i in range(len(ColumnName)):
    list_df2 = pd.DataFrame({''f'{ColumnName[i]}':   (df[ColumnName[i]].value_counts()) ,
                          '% Samples  ': round(100* (df[ColumnName[i]].value_counts())/(sum(df[ColumnName[i]].value_counts())),2),
                          ' total': sum(df[ColumnName[i]].value_counts()),
                          'stroke':  df_stroke[ColumnName[i]].value_counts(),
                           '%stroke': round(100*(df_stroke[ColumnName[i]].value_counts())/ (df[ColumnName[i]].value_counts()),2)

                           })
    
    plot_df2 = pd.DataFrame({'%stroke': round(100*(df_stroke[ColumnName[i]].value_counts())/ (df[ColumnName[i]].value_counts()),2)})
    
    print("")
    print(list_df2)
    plot_df2.plot(kind='bar', ylabel = '%')
    plt.ylabel('Counts')
    plt.title(f'{ColumnName[i]}')
    plt.show()

I see columns not only bmi have missing & unknown value

* gender: 1 case => missing
* smoking status: 1544 cases => unknown
* bmi: 201 cases => missing

In [ ]:
df.head()

# Let's prepare data for create Dicision Tree

Setting string value to be numerical values 

In [ ]:
def age_values(age_values):
  if age_values == 'none':
    return 0
  elif age_values == 'youth (< 9Y)' :
    return 1
  elif age_values == 'gen_z (9-25Y)' :
    return 2
  elif age_values == 'Millennials (25-41Y)':
    return 3
  elif age_values == 'gen_x (41-57Y)':
    return 4
  elif age_values == 'boomers_ii (57-67Y)':
    return 5
  elif age_values == 'boomer_i (67-76Y)' :
    return 6
  elif age_values == 'post_war (76-94Y)':
    return 7
  elif age_value == 'ww_ii (94Y <)':
    return 8
  else:
    return 'none'

def glucose_values(glucose_values):
    if glucose_values == 'none':
      return 0
    elif glucose_values =='server_hypoglycemia (< 53)' :
      return 1
    elif glucose_values == 'hypoglycemia (53-70)':
      return 2
    elif glucose_values == 'normal (70-125)' :
      return 3
    elif glucose_values == 'high (125-200)':
      return 4
    elif  glucose_values =='metabolic_consequences (200 <)':
      return 5
    else:
      return 'none'

def bmi_values(bmi_values):
    if bmi_values == 'none':
      return 0
    elif bmi_values == 'underweight (< 18.5)' :
      return 1
    elif bmi_values == 'normal (18.5-25)':
      return 2
    elif bmi_values =='overweight (25-30)':
      return 3
    elif  bmi_values == 'obese 30 <':
      return 4
    else:
      return 'none'

def gender_value(gender_value):
  if gender_value == 'Other':
    return 0
  elif gender_value =='Male':
    return 1
  elif gender_value =='Female':
    return 2
  else:
    return 'none'

def ever_married1(ever_married1):
  if ever_married1 == 'No':
    return 0
  elif ever_married1 == 'Yes':
    return 1
  else:
      return 'none'
  
def work_type1(work_type1):
  if work_type1 == 'Never_worked':
    return 0
  elif work_type1 == 'children':
    return 1
  elif work_type1 == 'Self-employed':
    return 2
  elif work_type1 == 'Govt_job':
    return 3
  elif work_type1 == 'Private':
    return 4
  else:
    return 'none'
 
def residence_type1(residence_type1):
  if residence_type1 == 'Urban':
    return 0
  elif residence_type1 == 'Rural':
    return 1
  else:
    return 'none'
  
def smoking_value(smoking_value):
  if smoking_value == 'Unknown':
    return 0
  elif smoking_value == 'never smoked':
    return 1
  elif smoking_value == 'formerly smoked':
    return 2
  elif smoking_value == 'smokes':
    return 3
  else:
    return 'none'

df['age_class1'] = df['age_class'].apply(age_values)
df['glucose_class1'] = df['glucose_class'].apply(glucose_values)
df['bmi_class1']=df['bmi_class'].apply(bmi_values)
df['gender_value']=df['gender'].apply(gender_value)
df['ever_married1'] = df['ever_married'].apply(ever_married1)
df['work_type1'] = df['work_type'].apply(work_type1)
df['residence_type1'] = df['Residence_type'].apply(residence_type1)
df['smoking_value'] = df['smoking_status'].apply(smoking_value)
df.head()

Now I have aded numeric columns representing string columns as below:

* age_class1
* glucose_class1
* bmi_class1
* gender_value
* ever_married1
* work_type1
* residence_type1
* smoking_value

In [ ]:
df.columns

In [ ]:
df.shape

I check numeric column by correlation check.

In [ ]:
ColumnName1 = ['hypertension', 'heart_disease',
       'age_class', 'glucose_class', 'bmi_class',
       'age_class1', 'glucose_class1', 'bmi_class1', 'gender_value',
       'ever_married1', 'work_type1', 'residence_type1', 'smoking_value']
sns.heatmap(df[ColumnName1].corr(),cmap='YlGnBu',annot=True)

# Set Prediction target and features 

In [ ]:
y = df.stroke
# Select Features 
features = ['hypertension', 'heart_disease',
       'age_class1', 'glucose_class1', 'bmi_class1', 'gender_value',
       'ever_married1', 'work_type1', 'residence_type1', 'smoking_value']
X = df[features]

I will create Decision tree classifier for overall.

In [ ]:
#splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.4, random_state = 39)   #40% test and 60% training

# Create the response for test dataset
dtree = DecisionTreeClassifier(max_depth=4).fit(X_train, y_train)
y_pred = dtree.predict(X_test)

#Evaluate model: model accuracy, how often is the classifier correct?
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))

In [ ]:
labels = X.columns
graph = Source(tree.export_graphviz(dtree ,feature_names = labels, max_depth = 4, filled = True))
display(SVG(graph.pipe(format='svg')))

from previouse, I know about 

* 1544 unknowns of Smoking_status, 

* 201 missing value of bmi 

* and 1 other of gender


They are set value zero number in each columns: 'unknowns of smoking_value' , 'missing value of bmi and other of gender

I will consider for each leave nodes.

# From Decision Tree above: I can divide them in 2 groups

# Group 1: age class1 in rank less than 5.5
it means: **Group of younger than 67 years**

* age class1 rank less than 4.5 => younger than 57 years
* age class1 rank between 4.5 - 5.5, it mean age 57-67 years this group inclues bmi class1 <= 0.5 that is missing value. 

so I will remove it and prediction, again.

* 1. sort age in range of less than 67 years
* 2. remove bmi_class1 =  0, it is missing value 

In [ ]:
df_age_gp1 = df[df['age'] < 67]
df_age_bmi_gp1 = df_age_gp1[df_age_gp1['bmi_class1'] > 0]

In [ ]:
# Prediction target  ### Classifier
y_gp1 = df_age_bmi_gp1.stroke
# Select Features 
features = ['hypertension', 'heart_disease',
       'age_class1', 'glucose_class1', 'bmi_class1', 'gender_value',
       'ever_married1', 'work_type1', 'residence_type1', 'smoking_value']
X_gp1 = df_age_bmi_gp1[features]
X_train_gp1, X_test_gp1, y_train_gp1, y_test_gp1 = train_test_split(X_gp1, y_gp1, test_size=0.3, random_state=61)

dtree_gp1 = DecisionTreeClassifier(max_depth=4).fit(X_train_gp1, y_train_gp1)

y_pred_gp1 = dtree_gp1.predict(X_test_gp1)


predict_gp1 = pd.DataFrame(index=y_test_gp1.index)
predict_gp1['stroke_actual'] = y_test_gp1
predict_gp1['Predict'] = y_pred_gp1


# As below is result  of Group 1 (age is less than 67 years).

In [ ]:

print('')
print(confusion_matrix(y_test_gp1,y_pred_gp1))
print(classification_report(y_test_gp1,y_pred_gp1))

predict_gp1.head()

# Group 2: age class1 in rank more than 5.5

 it means: **Group of 67 years or more**
 
 Also, this group inclues bmi class1 <= 0.5 that is missing value. 

so I will remove it and prediction, again.

* 1. sort age in range of 67 years or more.
* 2. remove bmi_class1 =  0, it is missing value 

In [ ]:
df_age_gp2 = df[df['age'] >= 67]
df_age_bmi_gp2 = df_age_gp2[df_age_gp2['bmi_class1'] > 0]

In [ ]:
# Prediction target  ### Classifier
y_gp2 = df_age_bmi_gp2.stroke
# Select Features 
features = ['hypertension', 'heart_disease',
       'age_class1', 'glucose_class1', 'bmi_class1', 'gender_value',
       'ever_married1', 'work_type1', 'residence_type1', 'smoking_value']
X_gp2 = df_age_bmi_gp2[features]

X_train_gp2, X_test_gp2, y_train_gp2, y_test_gp2 = train_test_split(X_gp2, y_gp2, test_size=0.3, random_state=33)

dtree_gp2 = DecisionTreeClassifier(max_depth=4).fit(X_train_gp2, y_train_gp2)


y_pred_gp2 = dtree_gp2.predict(X_test_gp2)


predict_gp2 = pd.DataFrame(index=y_test_gp2.index)
predict_gp2['actual'] = y_test_gp2
predict_gp2['Predict'] = dtree_gp2.predict(X_test_gp2)

# As below is result  of Group 2 (age 67 years or more)

In [ ]:

print('')
print(confusion_matrix(y_test_gp2,y_pred_gp2))
print(classification_report(y_test_gp2,y_pred_gp2))
predict_gp2.head()